4주차 예습과제에서는 5주차 내용인 GPT의 생성 능력을 맛보고자 합니다. 사용할 모델은 koalpaca라는 모델인데, 한국어 버전의 alpaca 모델입니다.

alpaca는 llama라는 모델을 instruction fine-tuning 해서 사람의 지시를 잘 따르는 모델입니다.

llama라는 모델은 LLaMA-13B의 경우 GPT-3보다 10배이상 작음에도 불구하고 대부분의 평가서 GPT-3보다 우수한 성능을 보이며, 더 나아가 LLaMA-65B의 경우 대부분의 벤치마크에서 Chinchilla, Gopher, GPT-3, PaLM와 유사하거나 더 뛰어난 결과를 보이는 훌륭한 모델입니다.

그러면 지금부터 koalpaca로 생성 모델을 한번 경험해보도록 하겠습니다!

In [1]:
# GPU 켜져 있는지 확인

!nvidia-smi

Wed Aug  6 06:08:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             44W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.tom

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

peft는 parameter efficient fine-tuning의 약자로, 큰 모델을 코랩에서 돌릴 수 있도록 쪼개서 만든 것입니다.

오래 걸립니다!

In [4]:
from peft import PeftModel, PeftConfig

In [5]:
peft_model_id = "beomi/qlora-koalpaca-polyglot-12.8b-50step"
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/428 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 28 files:   0%|          | 0/28 [00:00<?, ?it/s]

...

adapter_model.bin:   0%|          | 0.00/26.2M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=15360, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=8,

next token generation

In [6]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 난제를 해결할 수 있습니다. ​AI는 기계가 인간의 인식을 이해하고, 공감하고, 새로운 문제를 해결하는 데에 도움을 줍니다. ​AI는 사람처럼 보고


 QA

In [7]:
def gen(x):
    q = f"### 질문: {x}\n\n### 답변:"
    # print(q)
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=50,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [8]:
gen('건강하게 살기 위한 세 가지 방법은?')

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: 세 가지 방법이 있습니다.. 첫번째는 음식을 먹음으로써 스트레스를 해소하는것입니다. 두번째는 일을 찾아서 운동을 하는것입니다. 그리고 세번째는 운동을 해서 건강한 사람


top_k와 같은 parameter를 조절하면서 실험해보세요!

In [9]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64, top_k=50)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 숙제에 도전하는 기계를 의미한다. 이 과제는 크게 두 가지가 있는데 하나는 '인공지능'을 통해 인류의 삶의 질을 향상시키는 것이고, 다른 하나는


In [12]:
prompt = 'nlp란 무엇인가'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=256, top_k=50)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


nlp란 무엇인가? ​NLP란 Neuro-Linguistic Programs의 약자로써 인간의 자율신경계를 이용하여 내담자가 스스로 문제해결을 할 수 있도록 도와주는 신경언어프로그래밍입니다. 미국의 언어학자이자 인지치료의 선구자이며 신경언어프로그래밍의 창시자로 알려진 존. 밀턴(John Milton)박사에 의해서 개발되었습니다. 밀턴박사는 인간의 언어, 사고, 정서, 그리고 행동의 표현은 아주 강력한 인간의 신경체계에 의해 형성된다고 보았습니다. 따라서 Neuro-Linguistic Programs을 우리말로 번역하면 “신경언어프로그램 또는 신경언어장치” 정도가 됩니다. NLP의 원리 신경언어프로그래밍을 개발한 밀턴박사는 개인의 생각이나 느낌 그리고 행동이 말이나 글로 표현되고 그리고 그것들이 반복적으로 사용되어 습관이 되는 것처럼, 인간의 뇌와 조직도 역시 언어적 신경(linguistic neurons)에 의해 신호나 정보의 형태로 처리하는데 사용된다고 하였습니다. 이러한 언어적 신호


In [14]:
prompt = '이건 첫 번째 레슨, 좋은 건 너만 알기'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=256, top_k=50)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


이건 첫 번째 레슨, 좋은 건 너만 알기~"로 넘어가기 전에 그 전까지 내용에 있던 것 중 가장 중요한 것들을 복습하는 시간을 가졌어요.​- 그 다음 "좋은 건 너만 알기~"이 파트에서는 "나를 나답게 만드는 것", "나만의 스토리를 만드는 방법", "스토리를 만드는 방법"이라는 소제목을 가지고 레슨이 진행되었는데, "나를 나답게 만드는 것"을 시작으로 다양한 나의 이야기를 쓰고, 그 속에서 나를 찾아가는 방법을 배웠답니다.​- 그 다음 "스토리를 만드는 방법"에서는 스토리란 무엇인가에 대해 시작해 우리의 인생사를 토대로 나의 스토리를 만들어보고, 나의 스토리를 어떻게 써야 하는지에 대해 배우는 시간을 가졌어요. 그 뒤에는 선생님께서 나의 스토리를 듣고, 내가 만든 스토리를 토대로 내가 이야기를 써보는 시간을 가졌는데요. 그 때 썼던 내용을 토대로 스토리를 만들고 있는 나의 모습도 보여드릴게요.​ 나는 스토리를 만들면서 가장 중요


In [16]:
prompt = '이건 첫 번째 레슨, 좋은 건 너만 알기'

for temp in [0.3, 0.8, 1.2]:
    for k in [10, 40, 70, 100]:
        print(f"\n--- Temperature: {temp}, Top-k: {k} ---")
        with torch.no_grad():
            tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
            gen_tokens = model.generate(
                tokens,
                do_sample=True,
                temperature=temp,
                max_length=256,
                top_k=k
            )
            generated = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0]
            print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



--- Temperature: 0.3, Top-k: 10 ---


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


이건 첫 번째 레슨, 좋은 건 너만 알기 있기 없기?​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​

--- Temperature: 0.3, Top-k: 40 ---


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

이건 첫 번째 레슨, 좋은 건 너만 알기! (Duel)​이 노래가 가장 인상깊은 이유를 고르라면, 가사와 음을 제외하고 가장 큰 부분을 차지하는 건 바로 '듀에얼'이 아닐까?​'듀에얼'이란, '맞서 싸운다' 라는 뜻이다.가사는 대충 이렇다.​'네 마음을 얻기 위해선 내 마음부터 얻어야 해.''나한테 잘해줘, 안그러면 난 너 안 만나''난 너를 위해서라면 뭐든지 할거야'​이런 느낌의 가사.​나는 처음 이 노래를 들었을 때 가사보다 노래에 더 끌렸다.뭔가 신나서.이 노래는 가사와 음이 딱 들어맞아 듣기 편하기도 하고, 신나기도 하며, '듀에얼'의 가사가 참 인상깊었다.​가사에서 내 눈을 잡은 건 바로 이 '듀엣'이라는 부분.듀엣은 두 사람이 함께 화음을 쌓아 부르는 걸 뜻한다.​노래의 화음이 참 잘 어울렸다.​그리고 이 노래의 화자는 '네 사랑을 쟁취하

--- Temperature: 1.2, Top-k: 40 ---


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


이건 첫 번째 레슨, 좋은 건 너만 알기.. ♡ @eunzil_jin 님의 Instagram 동영상 • 2017 10월 29 오전 3:49 게시물 13.1천개. #firstglib #firstgliblive #beatinmyface #trunknwave #trunknwakeys #libsyn #helloworld #bythewaywww.instagram.com ​첫 번째 Live는 이 앨범 최초 공개 라이브로, 제가 좋아하는, 아니 모든 이가 좋아하는 First glib 라이브로 열었었죠. 👏🏼👏🏼👏🏼​ https://youtu.be/hZWgHn8ehXQ Tracy Childs x Samsmith (OVERFX Single): Touch My Mind @ygsweikaer https://y

--- Temperature: 1.2, Top-k: 70 ---


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


이건 첫 번째 레슨, 좋은 건 너만 알기첫 번째 레슨 시작해볼게잘 들어봐, 잘 따라와야 해하나만 더 명심해, 난 네가 원했었던내가 만든 모든 걸 하나도 틀리지 말고완벽한 태도로 다 해줘야 해, 너는 그게 필요해근데 그 태도란 말 어렵지나 그거 노력이 필요해 난 그거 정말 익숙하지아직 나도 어려워 네가 말한 모든 걸 내가잘 따라와 줄지 정말 궁금하지만걱정 할 것 없어, 뭐부터 하지?우선 내 말투부터 바꾸려고 노력 해볼게그리고 뭐든 말을 시작하기 전까진미리 생각하는 습관 좀 들어볼래?그리고 넌 매일 내가 자길 좋아하는지매일 그렇게 묻고 있지, 묻지마안 그러는 게 더 나아, 말해 봐두 번쨰 레슨, 네 입술에 맞춰 널 안아줄게그래, 그 다음 날 난 집 앞에 널 바래다주고네 얼굴 봐가, 기분 좋은 아침이잖아그럼 난 널 이렇게 불러 볼게, 야아그러고 나면 나는 이렇게 얘기해주지 "널 생각하고 상상하는 여기 이 느낌이 내 기쁨이야"그럼 너는 또 물어보겠지

--- Temperature: 1.2, Top-k: 100 ---
이건 첫 번째 레슨, 좋은 건 너만 알기!"였는데 바로 다음 레슨은 왜 이 녀석의 반응이 미지근한지 고민하는 다른 단원에 돌입하잖아!?​덕분에 주말 저녁을 다 보내게 되었지만(1박 2일로 잡아두길 잘했어..!) 확실히 하고 나서 책을 읽는 건 많이 달라졌다. 이전까지는 머릿속으로 문장구조와 기본 문법 지식을 하나도 모르는 한글자 한글자 이해하려 했는데, 이제는 훨씬 속도감 있게 넘어가니까 확실히 머릿속에 남아, 내가 무슨 의도로 이러한 표현을 하고 싶게 됐는지 훨씬 잘 다가온다!​그래서 아주 기쁜 마음으로 집콕의 하루를 정리할까 한다. 내 하루에 잘했다고 한 마디라도 해줘야지 :) 사실 어제 밤에 잠이 오지 않아서 늦어진 오늘이고 좀 늦었지만 그래도 정말 행복한 마음이다 ♡♥ ​​​ ​


⋅ temperature : 확률 분포 조정 -> 높아질수록 드문 단어가 많이 선택됨

⋅ top-k : 선택지를 제한 -> 단어 후보가 늘어남

둘 다 높아지면 답변의 창의성이 높아지지만, 너무 높은 경우 이상한 답변이 출력될 수 있음